In [12]:
import sys
import os
import shutil
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','..')))
import numpy as np
import cv2
import pandas as pd
from src.preprocessing import preprocessData,thresholding 
from src.TotalCalculation import timeConvertion,SunPosition
from matplotlib import pyplot as plt
from src.ClassPrediction import prediction

In [13]:
image_directory = r'C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_Astropark_Chaingmai\2024-06\2024-06-08'
mask = cv2.imread(r'masks\mask_delete_5.png',cv2.IMREAD_GRAYSCALE)
start_date = '2024-01-01'
end_date = os.path.basename(image_directory)
location = [18.849417,98.9538]
days = timeConvertion().time_duration(start_date,end_date,include_end_date=True).days

LSTM = SunPosition.LSTM(time_zone_offset=7)
EoT = SunPosition.calculate_EoT(day=days)
TC = SunPosition.TimeCorrectionFactor(Longitude=location[1],LSTM=LSTM,EoT=EoT)
dec = SunPosition.declination(day=days)
suntime = SunPosition.DaytimeInfo(latitude=location[0],declination=dec,TC=TC)
sunrise1,sunset1 = SunPosition.DaytimeInfo(latitude=location[0],declination=dec,TC=TC)

In [12]:
import cv2
import json
import shutil 
import sys,os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','..')))
from src.preprocessing import thresholding, preprocessData
from src.TotalCalculation import timeConvertion, SunPosition
from src.ClassPrediction import prediction
import os

# Define the start and end dates
start_date = '2024-01-01'
end_date = '2024-09-20'
location = [18.849417, 98.9538]
json_file_path = r"C:\Users\ASUS\Documents\NARIT_internship_2024\NARIT_internship_2024\RBratio.json"
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)
factor = [data['Factor'],data['Factor_night']]
days = timeConvertion().time_duration(start_date, end_date, include_end_date=True).days

# Create SunPosition instance
LSTM = SunPosition.LSTM(time_zone_offset=7)
EoT = SunPosition.calculate_EoT(day=days)
TC = SunPosition.TimeCorrectionFactor(Longitude=location[1], LSTM=LSTM, EoT=EoT)
dec = SunPosition.declination(day=days)
suntime = SunPosition.DaytimeInfo(latitude=location[0], declination=dec, TC=TC)
sunrise1, sunset1 = SunPosition.DaytimeInfo(latitude=location[0], declination=dec, TC=TC)

def copy_categorical_day(image_directory, output_directory, mask_directory, classes:dict, mode):
    mask = cv2.imread(mask_directory, cv2.IMREAD_GRAYSCALE)
    mask = preprocessData().crop_center(mask, crop_size=570)
    # Load and preprocess images
    images, name = preprocessData().load_images_and_preprocess(path=image_directory, mask=mask, apply_crop_sun=False, size=570)
    final, _, _ = thresholding().RBratio(input=images, factor=factor, filename=name, Time_zone=7, sunrise=sunrise1, sunset=sunset1)
    decimal = [timeConvertion().datetime_to_decimal(time=timeConvertion().ticks_to_datetime(ticks=t, time_zone=7)) for t in name]
    filtering = lambda x: (x > sunrise1) & (x < sunset1)
    if mode == 'day':
        day_indices = [index for index, value in enumerate(decimal) if filtering(value)]
    else: 
        day_indices = [index for index, value in enumerate(decimal) if not filtering(value)]
    final_day = final[0:len(day_indices)]
    name_day = [name[i] for i in day_indices]
    percentage = [prediction().CloudRatio(i, mask=mask) for i in final_day]
    classifier = [prediction().classify_sky(i, r) for i, r in percentage]
    for Class,folder_name in classes.items():
        for i in range(len(classifier)):
            original = os.path.join(image_directory)
            shutil.copy2(original, output_directory)
def process_image_folders(main_directory):
    classes_map = {
        'Clear sky (0 oktas)': 'Clear',
        'Fewer clouds (1 okta)': 'Clear',
        'Few clouds (2 oktas)': 'Partly Cloudy',
        'Scatter (3 oktas)': 'Partly Cloudy',
        'Mostly Scatter (4 oktas)': 'Mostly Cloudy',
        'Partly Broken (5 oktas)': 'Mostly Cloudy',
        'Mostly Broken (6 oktas)': 'Cloudy',
        'Broken (7 oktas)': 'Cloudy',
        'Overcast (8 oktas)': 'Overcast',
    }
    for cloud_class, output_suffix in classes_map.items():
        output_directory = os.path.join(r'C:\Users\ASUS\Documents\NARIT_internship_data\Dataset\Image_data_Australia\Image_data_Day', output_suffix)
        os.makedirs(output_directory, exist_ok=True)  # Create the output directory if it doesn't exist
    for subdir, _, files in os.walk(main_directory):
        copy_categorical_day(subdir, output_directory, mask_directory=r'C:\Users\ASUS\Documents\NARIT_internship_2024\NARIT_internship_2024\masks\International observatories\mask_Australia.png', 
                                classes=cloud_class, mode='day')
        print(f"Processed {cloud_class} images from {subdir}")

# Define the main image directory
main_image_directory = r'C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_Chile\2024-07'
for i in os.listdir(main_image_directory):
    image_data_path = os.path.join(main_image_directory,i)
    process_image_folders(i)


In [1]:
import cv2
import json
import shutil
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','..')))
from src.preprocessing import thresholding, preprocessData
from src.TotalCalculation import timeConvertion, SunPosition
from src.ClassPrediction import prediction

# Define the start and end dates
start_date = '2024-01-01'
end_date = '2023-12-10'
location = [18.57364, 98.48198]
json_file_path = r"C:\Users\ASUS\Documents\NARIT_internship_2024\NARIT_internship_2024\RBratio.json"
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)
factor = [data['Factor'], data['Factor_night']]
days = timeConvertion().time_duration(start_date, end_date, include_end_date=True).days

# Create SunPosition instance
LSTM = SunPosition.LSTM(time_zone_offset=7)
EoT = SunPosition.calculate_EoT(day=days)
TC = SunPosition.TimeCorrectionFactor(Longitude=location[1], LSTM=LSTM, EoT=EoT)
dec = SunPosition.declination(day=days)
sunrise1, sunset1 = SunPosition.DaytimeInfo(latitude=location[0], declination=dec, TC=TC)

def copy_categorical_day(image_directory, output_directory, mask_directory, classes:dict, mode):
    mask = cv2.imread(mask_directory, cv2.IMREAD_GRAYSCALE)
    mask = preprocessData().crop_center(mask, crop_size=700)

    # Load and preprocess images
    images, name = preprocessData().load_images_and_preprocess(path=image_directory, mask=mask, apply_crop_sun=False, size=700)
    final = []
    for i in images:
        B,G,R = cv2.split(i)
        B = B + 1e-3
        RB = (R/B) * 0.83
        RB = cv2.convertScaleAbs(RB)
        i = cv2.bitwise_and(i,i,mask=RB)
        i = cv2.cvtColor(i,cv2.COLOR_RGB2GRAY)
        final.append(i)

    decimal = [timeConvertion().datetime_to_decimal(time=timeConvertion().ticks_to_datetime(ticks=t, time_zone=7)) for t in name]
    filtering = lambda x: (x > sunrise1) & (x < sunset1)
    
    if mode == 'day':
        day_indices = [index for index, value in enumerate(decimal) if filtering(value)]
        
    else: 
        day_indices = [index for index, value in enumerate(decimal) if not filtering(value)]
    
    final_day = [final[i] for i in day_indices]
    name_day = [name[i] for i in day_indices]
    
    # Calculate cloud coverage and classify the images
    percentage = [prediction().CloudRatio(i, mask=mask) for i in final_day]
    classifier = [prediction().classify_sky(i, r) for i, r in percentage]

    # For each classified image, move it to the appropriate class folder
    for idx, cloud_class in enumerate(classifier):
        if cloud_class in classes:
            # Define the target folder based on the classification
            target_folder = os.path.join(output_directory, classes[cloud_class])
            os.makedirs(target_folder, exist_ok=True)  # Ensure the folder exists
            
            # Move the file to the target folder
            image_filename = name_day[idx]
            source_path = os.path.join(image_directory, str(image_filename) + '.png')
            destination_path = os.path.join(target_folder, str(image_filename) + '.png')
            shutil.copy2(source_path, destination_path)

def process_image_folders(main_directory):
    # Define class labels to folder names
    classes_map = {
        'Clear sky (0 oktas)': 'Clear',
        'Fewer clouds (1 okta)': 'Clear',
        'Few clouds (2 oktas)': 'Partly Cloudy',
        'Scatter (3 oktas)': 'Partly Cloudy',
        'Mostly Scatter (4 oktas)': 'Mostly Cloudy',
        'Partly Broken (5 oktas)': 'Mostly Cloudy',
        'Mostly Broken (6 oktas)': 'Cloudy',
        'Broken (7 oktas)': 'Cloudy',
        'Overcast (8 oktas)': 'Overcast',
    }
    
    # Iterate through subdirectories and process images
    for subdir, _, _ in os.walk(main_directory):
        copy_categorical_day(subdir, r'C:\Users\ASUS\Documents\NARIT_internship_data\Dataset\Image_data_TNO\Image_data_Day', 
                             mask_directory=r'C:\Users\ASUS\Documents\NARIT_internship_2024\NARIT_internship_2024\masks\International observatories\mask_Chile.png', 
                             classes=classes_map, mode='day')
        print(f"Processed images from {subdir}")

# Define the main image directory
main_image_directory = r'C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_Chile\2024-07'
for i in os.listdir(main_image_directory):
    image_data_path = os.path.join(main_image_directory, i)
    process_image_folders(image_data_path)


Processed images from C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_Chile\2024-07\2024-07-01
Processed images from C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_Chile\2024-07\2024-07-02
Processed images from C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_Chile\2024-07\2024-07-03


: 

In [17]:
import cv2
import numpy as np

def check_image_shape(img: list):
    if not img or not isinstance(img, list):
        raise ValueError("Input must be a non-empty list of images.")
    shapes = [(i.shape[1], i.shape[0]) for i in img]  # (width, height)
    min_width = min(w for w, h in shapes)
    min_height = min(h for w, h in shapes)
    resize_img = []
    for i in img:
        resized = cv2.resize(i, (min_width, min_height))
        resize_img.append(resized)
    
    return resize_img, (min_width, min_height)

In [18]:
paths = [r'C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_TNO\2024-09\2024-09-01', 
         r'C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_TNO\2024-09\2024-09-03']

# Initialize an empty list to store the images
images = []

# Loop through each directory in the paths list
for path in paths:
    # Loop through each file in the current directory
    for filename in os.listdir(path):
        # Construct the full file path
        filepath = os.path.join(path, filename)
        img = cv2.imread(filepath)
        images.append(img)
        # Read the image from the fu

In [19]:
import numpy as np
resize = check_image_shape(images)

def copy_categorical(target_directory:str,mask_directory:str,mode:str,classes:str,suntime:list):
    mask = cv2.imread(r'C:\Users\ASUS\Documents\NARIT_internship_2024\NARIT_internship_2024\masks\mask_delete_5.png',cv2.IMREAD_GRAYSCALE)
    mask = preprocessData().crop_center(mask,crop_size=570)
    images,name = preprocessData().load_images_and_preprocess(path=r'C:\Users\ASUS\Documents\NARIT_internship_data\All_sky_camera_Astropark_Chaingmai\2024-06\2024-06-08',mask=mask,apply_crop_sun=True)
    final,_,_ = thresholding().RBratio(input=images,filename=name,Time_zone=7,sunrise=sunrise1,sunset=sunset1)
    filtering = lambda x : (x > suntime[0]) & (x < suntime[1])
    decimal = [timeConvertion().datetime_to_decimal(time=timeConvertion().ticks_to_datetime(ticks=t,time_zone=7)) for t in name]
    filename = []
    if mode == 'day':
        classes = []
        day_indices = [index for index, value in enumerate(decimal) if filtering(value)]
        final_day = final[0:len(day_indices)]
        name_day = [name[i] for i in day_indices]
        percentage = [prediction().CloudRatio(i,mask=mask) for i in final_day]
        classifier = [prediction().classify_sky(i,r) for i,r in percentage]
        for i in range(len(classifier)):
            if classifier[i] == classes:
                filename.append(name_day[i])
    if mode == 'night':
        night_indices = [index for index, value in enumerate(decimal) if not filtering(value)]
        final_night = final[0:len(night_indices)]
        name_night = [name[i] for i in night_indices]
        percentage = [prediction().CloudRatio(i,mask=mask) for i in final_night]
        classifier = [prediction().classify_sky(i,r) for i,r in percentage]
        for i in range(len(classifier)):
            if classifier[i] == classes:
                filename.append(name_night[i])
    return filename

name = copy_categorical(target_directory=image_directory,mask_directory=r'C:\Users\ASUS\Documents\NARIT_internship_2024\NARIT_internship_2024\masks\mask_delete_5.png',mode='day',classes='Partly Broken (5 oktas)',suntime=[sunrise1,sunset1])